In [4]:
import pandas as pd
import numpy as np
import os
import sys
import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Audio
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from tensorflow.keras.layers import Conv1D,Dense,Flatten, Conv2D, AveragePooling2D, BatchNormalization, Dropout, MaxPooling1D, GlobalAveragePooling1D, LSTM, Activation
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
import warnings

In [5]:
def noise(data):
    noise_amp = 0.035 * np.random.uniform() * np.amax(data)
    data = data + noise_amp * np.random.normal(size = data.shape[0])
    return data

def stretch(data, rate = 0.8):
    return librosa.effects.time_stretch(data, rate = rate)

def shift(data):
    shift_range = int(np.random.uniform(low = -5, high = 5) * 1000)
    return np.roll(data, shift_range)

def pitch(data, sampling_rate, pitch_factor = 0.7):
    return librosa.effects.pitch_shift(data, sr = sampling_rate, n_steps = pitch_factor)



In [6]:
def extract_features(data,sample_rate):
    # ZCR
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result = np.hstack((result, zcr)) # stacking horizontally

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft)) # stacking horizontally

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc)) # stacking horizontally

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms)) # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel)) # stacking horizontally
    
    return result

def get_features(path):
    # duration and offset are used to take care of the no audio in start and the ending of each audio files as seen above.
    data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)
    
    # without augmentation
    res1 = extract_features(data,sample_rate)
    result = np.array(res1)
    
    # data with noise
    noise_data = noise(data)
    res2 = extract_features(noise_data,sample_rate)
    result = np.vstack((result, res2)) # stacking vertically
    
    # data with stretching and pitching
    new_data = stretch(data)
    data_stretch_pitch = pitch(new_data, sample_rate)
    res3 = extract_features(data_stretch_pitch,sample_rate)
    result = np.vstack((result, res3)) # stacking vertically
    
    return result


In [29]:
df=pd.read_csv('df.csv')

In [30]:
X, Y = [], []
c=0
for path, emotion in zip(df.Files, df.Emotions):
    feature = get_features(path)
    for item in feature:
        X.append(item)
        # appending emotion 3 times as we have made 3 augmentation techniques on each audio file.
        Y.append(emotion)
    c=c+1
    if c==1:
        break

In [32]:
len(X)

3

In [33]:
Features = pd.DataFrame(X)
Features['labels'] = Y

In [34]:
Features

,0,1,2,3,4,5,6,7,8,9,...,153,154,155,156,157,158,159,160,161,labels
0,0.321275,0.729664,0.750032,0.730624,0.735275,0.713529,0.660531,0.684966,0.733049,0.753972,...,4.310903e-06,3.291511e-06,2.148075e-06,2.279739e-06,5.116493e-06,8.190282e-06,0.000007,0.000005,4.245834e-07,neutral
1,0.311519,0.798504,0.813604,0.804001,0.805637,0.793914,0.680412,0.675899,0.719832,0.768509,...,3.684054e-05,3.639058e-05,3.532505e-05,3.433467e-05,3.925696e-05,3.911997e-05,0.000039,0.000039,3.294042e-05,neutral
2,0.188256,0.622128,0.699219,0.753332,0.721223,0.701737,0.682353,0.662828,0.686490,0.733966,...,8.576332e-07,9.576413e-07,7.733594e-07,5.233102e-07,3.592796e-07,9.261687e-07,0.000002,0.000001,7.753984e-08,neutral


In [143]:
# Features.to_csv('features3.0.csv', index=False)
Features=pd.read_csv('features3.0.csv')



In [144]:
# X = Features.iloc[: ,:-1].values
Y = Features['labels'].values
encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()
# splitting data
# x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0, shuffle=True)
# x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [145]:
sentiment_key={0:'angry',1:'calm',2:'disgust',3:'fear',4:'happy',5:'neutral',6:'sad',7:'surprise'}

In [146]:
list=[]
for i,mapping in enumerate(original_categories):
    tup=(Y[i],mapping[0])
    if i%17==0:
        print(tup)

(array([0., 0., 0., 0., 0., 1., 0., 0.]), 'neutral')
(array([0., 1., 0., 0., 0., 0., 0., 0.]), 'calm')
(array([0., 1., 0., 0., 0., 0., 0., 0.]), 'calm')
(array([0., 0., 0., 0., 1., 0., 0., 0.]), 'happy')
(array([0., 0., 0., 0., 0., 0., 1., 0.]), 'sad')
(array([1., 0., 0., 0., 0., 0., 0., 0.]), 'angry')
(array([1., 0., 0., 0., 0., 0., 0., 0.]), 'angry')
(array([0., 0., 0., 1., 0., 0., 0., 0.]), 'fear')
(array([0., 0., 1., 0., 0., 0., 0., 0.]), 'disgust')
(array([0., 0., 1., 0., 0., 0., 0., 0.]), 'disgust')
(array([0., 0., 0., 0., 0., 0., 0., 1.]), 'surprise')
(array([0., 0., 0., 0., 0., 1., 0., 0.]), 'neutral')
(array([0., 1., 0., 0., 0., 0., 0., 0.]), 'calm')
(array([0., 0., 0., 0., 1., 0., 0., 0.]), 'happy')
(array([0., 0., 0., 0., 1., 0., 0., 0.]), 'happy')
(array([0., 0., 0., 0., 0., 0., 1., 0.]), 'sad')
(array([1., 0., 0., 0., 0., 0., 0., 0.]), 'angry')
(array([0., 0., 0., 1., 0., 0., 0., 0.]), 'fear')
(array([0., 0., 0., 1., 0., 0., 0., 0.]), 'fear')
(array([0., 0., 1., 0., 0., 0.

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [37]:
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((2, 162, 1), (2, 1), (1, 162, 1), (1, 1))

In [38]:
x_train.shape

(2, 162, 1)

# arch-1

In [41]:
model = Sequential()
model.add(Conv1D(64, 5, activation='relu', input_shape=(x_train.shape[1], 1)))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 158, 64)           384       
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 52, 64)           0         
 1D)                                                             
                                                                 
 conv1d_7 (Conv1D)           (None, 48, 128)           41088     
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 16, 128)          0         
 1D)                                                             
                                                                 
 conv1d_8 (Conv1D)           (None, 12, 128)           82048     
                                                                 
 global_average_pooling1d_1   (None, 128)             

In [42]:
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=2, min_lr=0.0000001)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

In [43]:
history=model.fit(x_train, y_train, batch_size=16, epochs=50, validation_data=(x_test, y_test), callbacks=[es, rlrp])

Epoch 1/50
1446/1446 [==============================] - 9s 6ms/step - loss: 1.8374 - accuracy: 0.2609 - val_loss: 1.7519 - val_accuracy: 0.3073 - lr: 1.0000e-04
Epoch 2/50
1446/1446 [==============================] - 9s 6ms/step - loss: 1.7307 - accuracy: 0.3046 - val_loss: 1.6922 - val_accuracy: 0.3202 - lr: 1.0000e-04
Epoch 3/50
1446/1446 [==============================] - 8s 6ms/step - loss: 1.6823 - accuracy: 0.3161 - val_loss: 1.6411 - val_accuracy: 0.3443 - lr: 1.0000e-04
Epoch 4/50
1446/1446 [==============================] - 8s 6ms/step - loss: 1.6483 - accuracy: 0.3308 - val_loss: 1.6140 - val_accuracy: 0.3471 - lr: 1.0000e-04
Epoch 5/50
1446/1446 [==============================] - 8s 6ms/step - loss: 1.6219 - accuracy: 0.3438 - val_loss: 1.5868 - val_accuracy: 0.3635 - lr: 1.0000e-04
Epoch 6/50
1446/1446 [==============================] - 9s 6ms/step - loss: 1.5941 - accuracy: 0.3588 - val_loss: 1.5589 - val_accuracy: 0.3741 - lr: 1.0000e-04
Epoch 7/50
1446/1446 [============

# arch-2

In [47]:

# Define the CRNN model
model = Sequential()

# Convolutional layers for feature extraction
model.add(Conv1D(64, 3, activation='relu', input_shape=(x_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(256, 3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

# Recurrent layer for temporal modeling
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(128))

# Fully connected layers for classification
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # Assuming 8 classes

# Compile the model
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Display model summary
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_12 (Conv1D)          (None, 160, 64)           256       
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 80, 64)           0         
 1D)                                                             
                                                                 
 dropout_7 (Dropout)         (None, 80, 64)            0         
                                                                 
 conv1d_13 (Conv1D)          (None, 78, 128)           24704     
                                                                 
 max_pooling1d_10 (MaxPoolin  (None, 39, 128)          0         
 g1D)                                                            
                                                                 
 dropout_8 (Dropout)         (None, 39, 128)          

In [48]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=2, min_lr=0.0000001)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

In [ ]:
history=model.fit(x_train, y_train, batch_size=16, epochs=200, validation_data=(x_test, y_test), callbacks=[es, rlrp])

Epoch 1/200
1446/1446 [==============================] - 44s 31ms/step - loss: 1.7422 - accuracy: 0.2962 - val_loss: 1.6607 - val_accuracy: 0.3349 - lr: 1.0000e-04
Epoch 2/200
1446/1446 [==============================] - 63s 44ms/step - loss: 1.6710 - accuracy: 0.3235 - val_loss: 1.5840 - val_accuracy: 0.3583 - lr: 1.0000e-04
Epoch 3/200
1446/1446 [==============================] - 69s 47ms/step - loss: 1.6174 - accuracy: 0.3418 - val_loss: 1.5382 - val_accuracy: 0.3790 - lr: 1.0000e-04
Epoch 4/200
1446/1446 [==============================] - 66s 46ms/step - loss: 1.5665 - accuracy: 0.3623 - val_loss: 1.4839 - val_accuracy: 0.3961 - lr: 1.0000e-04
Epoch 5/200
1446/1446 [==============================] - 67s 46ms/step - loss: 1.5303 - accuracy: 0.3808 - val_loss: 1.4479 - val_accuracy: 0.4062 - lr: 1.0000e-04
Epoch 6/200
1446/1446 [==============================] - 64s 44ms/step - loss: 1.4974 - accuracy: 0.3954 - val_loss: 1.4221 - val_accuracy: 0.4226 - lr: 1.0000e-04
Epoch 7/200
1446

Epoch 51/200
 945/1446 [==================>...........] - ETA: 42s - loss: 1.1818 - accuracy: 0.5343

# arch - 3

In [12]:
model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(x_train.shape[1],1))) 
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(8)) # Target class number
model.add(Activation('softmax'))
# opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
opt = keras.optimizers.Adam(learning_rate=0.0001)
# opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 162, 256)          2304      
                                                                 
 activation (Activation)     (None, 162, 256)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 162, 256)          524544    
                                                                 
 batch_normalization (BatchN  (None, 162, 256)         1024      
 ormalization)                                                   
                                                                 
 activation_1 (Activation)   (None, 162, 256)          0         
                                                                 
 dropout (Dropout)           (None, 162, 256)          0         
                                                        

In [13]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [14]:
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=2, min_lr=0.0000001)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

history=model.fit(x_train, y_train, batch_size=16, epochs=200, validation_data=(x_test, y_test), callbacks=[es, rlrp])

Epoch 1/200
1446/1446 [==============================] - 187s 129ms/step - loss: 1.5047 - accuracy: 0.4052 - val_loss: 1.4096 - val_accuracy: 0.4566 - lr: 1.0000e-04
Epoch 2/200
1446/1446 [==============================] - 220s 152ms/step - loss: 1.3201 - accuracy: 0.4777 - val_loss: 1.3386 - val_accuracy: 0.4800 - lr: 1.0000e-04
Epoch 3/200
1446/1446 [==============================] - 168s 116ms/step - loss: 1.2513 - accuracy: 0.5029 - val_loss: 1.3086 - val_accuracy: 0.4942 - lr: 1.0000e-04
Epoch 4/200
1446/1446 [==============================] - 169s 117ms/step - loss: 1.2048 - accuracy: 0.5229 - val_loss: 1.2724 - val_accuracy: 0.5060 - lr: 1.0000e-04
Epoch 5/200
1446/1446 [==============================] - 170s 117ms/step - loss: 1.1666 - accuracy: 0.5401 - val_loss: 1.2645 - val_accuracy: 0.5271 - lr: 1.0000e-04
Epoch 6/200
1446/1446 [==============================] - 172s 119ms/step - loss: 1.1238 - accuracy: 0.5575 - val_loss: 1.1895 - val_accuracy: 0.5441 - lr: 1.0000e-04
Epoc

In [33]:
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=2, min_lr=0.0000001)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

history=model.fit(x_train, y_train, batch_size=16, epochs=200, validation_data=(x_test, y_test), callbacks=[es, rlrp])

Epoch 1/200
1446/1446 [==============================] - 171s 118ms/step - loss: 1.5221 - accuracy: 0.3965 - val_loss: 1.4929 - val_accuracy: 0.4362 - lr: 1.0000e-04
Epoch 2/200
1446/1446 [==============================] - 240s 166ms/step - loss: 1.3263 - accuracy: 0.4745 - val_loss: 1.3919 - val_accuracy: 0.4540 - lr: 1.0000e-04
Epoch 3/200
1446/1446 [==============================] - 239s 165ms/step - loss: 1.2533 - accuracy: 0.5023 - val_loss: 1.4205 - val_accuracy: 0.4675 - lr: 1.0000e-04
Epoch 4/200
1446/1446 [==============================] - 251s 174ms/step - loss: 1.2097 - accuracy: 0.5199 - val_loss: 1.2986 - val_accuracy: 0.4778 - lr: 1.0000e-04
Epoch 5/200
1446/1446 [==============================] - 239s 165ms/step - loss: 1.1645 - accuracy: 0.5424 - val_loss: 1.2551 - val_accuracy: 0.5147 - lr: 1.0000e-04
Epoch 6/200
1446/1446 [==============================] - 266s 184ms/step - loss: 1.1273 - accuracy: 0.5550 - val_loss: 1.1881 - val_accuracy: 0.5459 - lr: 1.0000e-04
Epoc

In [15]:
from tensorflow.keras.models import load_model
model.save('model_2.h5')

# arch-4

In [10]:
model = Sequential()

# Input layer
model.add(Conv1D(256, 8, padding='same',input_shape=(x_train.shape[1],1))) 
model.add(Activation('relu'))

model.add(Conv1D(64, 3, padding='same'))#, input_shape=(x_train.shape[1], 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Convolutional layers
model.add(Conv1D(128, 3, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv1D(256, 3, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv1D(512, 3, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))

# Global average pooling and dense layers
model.add(GlobalAveragePooling1D())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_9 (Conv1D)           (None, 162, 256)          2304      
                                                                 
 activation_9 (Activation)   (None, 162, 256)          0         
                                                                 
 conv1d_10 (Conv1D)          (None, 162, 64)           49216     
                                                                 
 batch_normalization_8 (Batc  (None, 162, 64)          256       
 hNormalization)                                                 
                                                                 
 activation_10 (Activation)  (None, 162, 64)           0         
                                                                 
 conv1d_11 (Conv1D)          (None, 162, 128)          24704     
                                                      

In [11]:
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=2, min_lr=0.0000001)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

history=model.fit(x_train, y_train, batch_size=16, epochs=200, validation_data=(x_test, y_test), callbacks=[es, rlrp])

Epoch 1/200
 683/1446 [=============>................] - ETA: 56s - loss: 1.8423 - accuracy: 0.2496

KeyboardInterrupt: 

In [120]:
from tensorflow.keras.models import load_model

In [2]:
model=load_model('model_2.h5')

In [7]:
sample_path='sample/25 Sep, 15.18.wav'

In [45]:
data, sample_rate = librosa.load(sample_path, duration=2.5, offset=0.6)
sample_features=extract_features(data,sample_rate)

In [49]:
data, sample_rate = librosa.load(sample_path, duration=2.5, offset=0.6)
sample_features=extract_features(data,sample_rate)
sample_features=sample_features.reshape(1,162,1)
sample_features.shape
result=model.predict(sample_features)

1/1 [==============================] - 0s 16ms/step


In [ ]:
result.argmax(axis=1)

In [119]:
sentiment_key[2]

'disgust'

In [41]:
sample_features.shape

(162, 1)

In [25]:
model.predict(sample_features)

ValueError: in user code:

    File "C:\Users\rachi\anaconda3\lib\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\rachi\anaconda3\lib\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\rachi\anaconda3\lib\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\rachi\anaconda3\lib\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "C:\Users\rachi\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\rachi\anaconda3\lib\site-packages\keras\backend.py", line 6523, in pool2d
        x = tf.compat.v1.nn.max_pool(

    ValueError: Exception encountere